In [1]:
import os

from whoosh import highlight
from whoosh.index import create_in
from whoosh.fields import Schema, ID, TEXT
from whoosh.lang.morph_en import variations
from whoosh.qparser import QueryParser
from whoosh.query import *
import re

In [6]:
def morphology_tool():
    def build_index():
        schema = Schema(title=TEXT(stored=True), content=TEXT(stored=True),
                   path=ID(stored=True))

        if not os.path.exists('index'):
            os.mkdir('index')
        my_index = create_in('index', schema)

        OS_SEP = os.sep

        writer = my_index.writer()

        for document in os.listdir('corpus_of_british_fiction'):
            if document.endswith('.txt'):
                with open('corpus_of_british_fiction' + OS_SEP + document, 'r',encoding='UTF-8') as text:
                    writer.add_document(title=document, content=str(text.read()),
                                       path=document)
        writer.commit()
    
        user_query = input('which word are you looking for?: ')
        myquery = QueryParser('content', my_index.schema, termclass=Variations).parse(str(user_query))
    
        with my_index.searcher() as searcher:
            results = searcher.search(myquery, limit=None)
            results.fragmenter = highlight.ContextFragmenter(charlimit=None)
            return_results = []
            for index, hit in enumerate(results):
                return_results.append(hit.highlights('content'.strip()))
        return return_results
    
    def clean_sentence(sentence):
        whitespace = ['\n', '\t', '  ']
        multiple_spaces = ['  ', '   ', '    ', '     ']
        markup_tags = [r'\<b.*?>', r'\<\/b>']
        for char in whitespace + markup_tags:
            if char in markup_tags:
                sentence = re.sub(char, '', sentence)
            else:
                sentence = re.sub(char, ' ', sentence)
        for char in multiple_spaces:
            sentence = re.sub(char, ' ', sentence)
        return sentence
    
    def return_result(list_of_results):
        for sentence in return_results:
            print('\n', clean_sentence(sentence))
        return
    
    return_results = build_index()
    return_result(return_results)
    return

In [7]:
morphology_tool()

which word are you looking for?: think

 papa and mamma would think so too, if they thought much about it. However...if I can always think on these things, I think I might well love God...and an habitual thinker; and when he smiled

 cushioned seat, and thought, and thought, until the tears...me to say _I_ think otherwise. If I thought myself doomed to...and then, dreadful thought! she would think me cooled and changed
